# Open AI Quote Generator (Function Calling)

A simple quote generator to demonstrate function calling using Open AI.


In [7]:
import os

requirements_installed = False
max_retries = 3
retries = 0


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return

In [ ]:
install_requirements()

In [3]:
from dotenv import load_dotenv
import os


def setup_env():
    """Sets up the environment variables"""

    def check_env(env_var):
        value = os.getenv(env_var)
        if value is None:
            print(f"Please set the {env_var} environment variable.")
            exit(1)
        else:
            print(f"{env_var} is set.")

    load_dotenv()

    variables_to_check = ["OPENAI_API_KEY"]

    for var in variables_to_check:
        check_env(var)

In [ ]:
setup_env()

In [6]:
QUOTABLE_TOPICS = topics = [
    "inspirational",
    "motivational",
    "life",
    "love",
    "success",
    "happiness",
    "wisdom",
    "leadership",
    "friendship",
    "education",
    "family",
    "peace",
    "hope",
    "dreams",
    "creativity",
    "imagination",
    "confidence",
    "self-love",
    "self-care",
    "self-improvement",
    "self-esteem",
    "self-discipline",
    "self-respect",
    "self-confidence",
    "self-awareness",
    "self-compassion",
    "self-acceptance",
    "self-development",
    "self-motivation",
    "self-control",
    "self-growth",
    "self-belief",
    "self-worth",
    "self-help",
    "self-realization",
    "self-discovery",
    "self-reflection",
    "self-expression",
    "self-regulation",
    "self-empowerment",
    "self-actualization",
    "mindfulness",
]

In [8]:
# Reference: https://cookbook.openai.com/examples/function_calling_with_an_openapi_spec
from openai import OpenAI
import os
import pandas as pd
import random


class QuoteGenerator:
    """A simple quote generator using LLM that demonstrates tool use."""

    def __init__(self, quotes_db_path="data/quotes.csv"):
        self.api_key = os.getenv("OPENAI_API_KEY")
        self.openai = OpenAI(self.api_key)
        try:
            self.quotes_db = pd.read_csv(quotes_db_path)
        except FileNotFoundError:
            print(f"Quotes database not found at {quotes_db_path}.")
            empty_quotes = pd.DataFrame(columns=["quote"])
            empty_quotes.to_csv(quotes_db_path, index=False)
            self.quotes_db = empty_quotes

    def does_quote_exist(self, quote) -> bool:
        """Checks if a quote exists in the database"""
        return self.quotes_db["quote"].str.contains(quote).any()

    def add_quote(self, quote):
        """Adds a quote to the database"""
        if self.does_quote_exist(quote):
            print("Quote already exists in the database.")
            return
        self.quotes_db = self.quotes_db.append({"quote": quote}, ignore_index=True)
        self.quotes_db.to_csv("data/quotes.csv", index=False)
        print("Quote added successfully.")

    def get_random_topic(self):
        """Gets a random topic from the quotable topics"""
        return random.choice(QUOTABLE_TOPICS)

    def generate_quote(self):
        """Generates a quote using the LLM model"""
        topic = self.get_random_topic()
        prompt = f"""
            You are a quote generator AI that generates quotes for the user.
            Topic: {topic}
            Generate a quote about {topic}.
            Make sure the quote is unique and doesn't exist in the database.
            You have a tool to check if the quote already exists in the database.
        """
        tools = [
            {
                "type": "function",
                "function": {
                    "name": "does_quote_exist",
                    "parameters": {
                        "type": "object",
                        "properties": {"quote_to_check": {"type": "string"}},
                        "required": ["quote_to_check"],
                    },
                },
            }
        ]
        completion = self.openai.chat.completions.create(
            messages=[
                {"role": "system", "content": "You are a quote generator AI."},
                {"role": "user", "content": topic},
            ],
            model="gpt-4o",
            tools=tools,
        )

        tool_calls = completion.choices[0].message.tool_calls

        tool_output = ""

        for tool_call in tool_calls:
            tool_output += "\n" + tool_call.name + "\n"
            if tool_call.name == "does_quote_exist":
                quote_to_check = tool_call.parameters.quote_to_check
                quote_exists = self.does_quote_exist(quote_to_check)
                if quote_exists:
                    tool_output += (
                        f"The quote '{quote_to_check}' already exists in the database."
                    )
                else:
                    tool_output += (
                        f"The quote '{quote_to_check}' does not exist in the database."
                    )
        # TODO: Finish this function